Making/Desgining of the text_normalizer dependency, Which is been stored in the .py file, And we will use it in our main code

In [1]:
import spacy
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
import re
from bs4 import BeautifulSoup
from contractions import CONTRACTION_MAP
import unicodedata

nlp = spacy.load('en_core_web_sm')
tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')

2022-12-19 21:55:01.146034: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-19 21:55:01.270733: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-19 21:55:01.273696: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-19 21:55:01.273710: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [2]:
# Cleaning Text - Strip HTML
def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text()
    return stripped_text

In [3]:
# Removing accented characters
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

In [4]:
# Expanding Contractions
def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

In [5]:
# Removing Special Characters
def remove_special_characters(text):
    text = re.sub('[^a-zA-z0-9]', '', text)
    return text

In [6]:
# Lemmatizing Text
def lemmatize_text(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

In [7]:
# Removing Stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

In [8]:
# Normalize text corpus - tying it all together
def normalize_corpus(corpus, html_stripping=True, contraction_expansion=True, accented_char_removal=True, text_lower_case=True, text_lemmatization=True, special_char_removal=True, stopword_removal=True):
    normalize_corpus = []
    # Normalize each document in the corpus
    for doc in corpus:
        # Strip HTML
        if html_stripping:
            doc = strip_html_tags(doc)
        # Remove accented Characters
        if accented_char_removal:
            doc = remove_accented_chars(doc)
        # Expand contractions
        if contraction_expansion:
            doc = expand_contractions(doc)
        # Lowercase the text
        if text_lower_case:
            doc = doc.lower()
        # Removing extra newlines
        doc = re.sub(r'[\r|\n|\r\n]+', ' ', doc)
        # Insert spaces between special characters to isolate them
        special_char_pattern = re.compile(r'([{.(-)!}])')
        doc = special_char_pattern.sub(" \\1 ", doc)
        # lemmatize text
        if text_lemmatization:
            doc = lemmatize_text(doc)
        # Remove special characters
        if special_char_removal:
            doc = remove_special_characters(doc)
        # Remove extra whitespaces
        doc = re.sub(' +', ' ', doc)
        # Remove stopwords
        if stopword_removal:
            doc = remove_stopwords(doc, is_lower_case=text_lower_case)
            
        normalize_corpus.append(doc)
        
    return normalize_corpus

In [9]:
# Sample Demo
document = """
Héllo! Héllo! can you hear me! I just heard about Python!

\r\n 
              It's an amazing language which can be used for Scripting, Web development,\r\n\r\n
              Information Retrieval, Natural Language Processing, Machine Learning & Artificial Intelligence!\n
              What are you waiting for? Go and get started.
 He's learning, she's learning, they've already\n\n
              got a headstart!
           """

normalize_corpus([document], text_lemmatization=False, stopword_removal=False, text_lower_case=False)

['HelloHellocanyouhearmeIjustheardaboutPythonItisanamazinglanguagewhichcanbeusedforScriptingWebdevelopmentInformationRetrievalNaturalLanguageProcessingMachineLearningArtificialIntelligenceWhatareyouwaitingforGoandgetstartedHeislearningsheislearningtheyhavealreadygotaheadstart']

In [10]:
normalize_corpus([document])

['hellohellocanyouhearIIjusthearaboutpythonitbeanamazinglanguagewhichcanbeuseforscriptingwebdevelopmentinformationretrievalnaturallanguageprocessingmachinelearningartificialintelligencewhatbeyouwaitforgoandgetstarthebelearnshebelearntheyhavealreadygetaheadstart']